In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('recommender').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
data=spark.read.csv("/FileStore/tables/movielens_ratings.csv",header=True,inferSchema=True)

In [5]:
data.printSchema()

root
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)
-- userId: integer (nullable = true)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [7]:
data.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
 12| 2.0| 0|
 15| 1.0| 0|
 17| 1.0| 0|
 19| 1.0| 0|
 21| 1.0| 0|
 23| 1.0| 0|
 26| 3.0| 0|
 27| 1.0| 0|
 28| 1.0| 0|
 29| 1.0| 0|
 30| 1.0| 0|
 31| 1.0| 0|
 34| 1.0| 0|
 37| 1.0| 0|
 41| 2.0| 0|
+-------+------+------+
only showing top 20 rows

In [8]:
training,test=data.randomSplit([0.8,0.2])

In [9]:
als=ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [10]:
model=als.fit(training)

In [11]:
prediction=model.transform(test)

In [12]:
prediction.show()

+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 1.0| 26| 1.0418293|
 31| 3.0| 8| 1.8099262|
 31| 3.0| 7| 0.4875813|
 31| 2.0| 25|0.69047296|
 31| 1.0| 24| 2.705785|
 31| 1.0| 18| 1.5893286|
 85| 1.0| 12| 1.1672462|
 85| 1.0| 13| 1.2462916|
 85| 2.0| 20| 2.4897687|
 85| 1.0| 29| -2.806757|
 65| 1.0| 22| 1.1716416|
 65| 1.0| 16|0.38499063|
 65| 5.0| 23| 2.0363698|
 53| 3.0| 13| 1.063276|
 53| 5.0| 8| 3.4248312|
 53| 1.0| 7|-1.8512486|
 78| 1.0| 1| 0.8041982|
 78| 1.0| 13| 0.3441172|
 78| 1.0| 19|0.84074557|
 78| 1.0| 4| 0.8032946|
+-------+------+------+----------+
only showing top 20 rows

In [13]:
evaluator=RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [14]:
rmse=evaluator.evaluate(prediction)

In [15]:
print("RMSE")
print(rmse)

RMSE
1.9116457738740535

In [16]:
single_user=test.filter(test['userId']==11).select(['movieId','userId'])

In [17]:
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 9| 11|
 16| 11|
 19| 11|
 20| 11|
 21| 11|
 45| 11|
 47| 11|
 71| 11|
 76| 11|
 81| 11|
+-------+------+

In [18]:
recommendations=model.transform(single_user)

In [19]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 21| 11| 2.967926|
 16| 11| 2.0904775|
 76| 11| 1.9494681|
 47| 11| 1.9100499|
 19| 11| 1.1250762|
 20| 11| 1.0902946|
 81| 11| 0.9576685|
 71| 11| 0.57548034|
 45| 11| 0.273083|
 9| 11|-0.25650662|
+-------+------+-----------+